In [1]:
import os, sys
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
import numpy as np
import h5py
from keras.layers import Conv2D, BatchNormalization, Activation, Flatten, Dense, Dropout, LSTM, Input, TimeDistributed, Bidirectional, GlobalMaxPooling1D, MaxPooling1D, Permute, Reshape, Conv1D
from keras import initializers, Model, optimizers, callbacks
from keras.models import load_model
from keras.optimizers import SGD, RMSprop, SGD, Adam, Nadam
from keras.metrics import *
#from keras.utils.training_utils import multi_gpu_model
from glob import glob
from keras import optimizers
import tensorflow as tf
import pandas as pd
from pandas import Categorical

In [2]:
def import_hdf5(filename):
    f = h5py.File(filename,'r')
    a_group_key = list(f.keys())[0]
    data = list(f[a_group_key])
    return np.array(data)

In [3]:
from glob import glob

In [4]:
nomeschema='09'
npyfilespath ="../public_data/CHUNKS_09/"
MODELDIR='./MODELS/'
#os.chdir(npyfilespath)

In [5]:
TASK="HaLT-filt830-"
#subjects=['A','B','C','E','F','G','H','I','J','K','L','M']
subjects=['G']
searchstring=npyfilespath+TASK+"Subject"+subjects[0]+"*_DATA.hdf5"

In [6]:
glob(searchstring)

['../public_data/CHUNKS_09/HaLT-filt830-SubjectG1603016StLRHandLegTongue_chunks_09_DATA.hdf5',
 '../public_data/CHUNKS_09/HaLT-filt830-SubjectG1603226StLRHandLegTongue_chunks_09_DATA.hdf5',
 '../public_data/CHUNKS_09/HaLT-filt830-SubjectG1604126StLRHandLegTongue_chunks_09_DATA.hdf5']

In [19]:
tdNnum   = [ 24 , 24 , 24 ]
tdNact   = 'relu'
sdCnum   = [ 30 , 20 , 15 ]
sdCker   = [ 5 , 5 , 5  ]
sdCact   = 'relu'
poolSize  = [ 3 , 3 , 3  ]
pooltype  = 'ave'
LSnum = [ 0 , 0  ]
LSdrop = [ 0.1 , 0  ]
FNnum = [ 100 , 50  ]
FNact   = 'relu'
FNdrop = [ 0.5 , 0.2  ]

In [20]:
###  TRAINING PARAMETERS
patience_e = 150
patience_p = 50
factor = 0.2
batch_size = 256
# Define the optimizer
optimizer = RMSprop(lr=1e-4, decay=1e-5)
IDX=0;
SUB=subjects[IDX];
input_shape=(200,9)
NUMCATEGORIES=6
print(input_shape)

(200, 9)


In [21]:
from interleavedV4 import makeInterleavedModel, nameInterleavedModel
modelname = nameInterleavedModel(TASK=TASK,SUB=SUB,nomeschema=nomeschema
                                 ,lr=1e-4
                                 ,patience_e=patience_e,patience_p=patience_p,factor=factor
                                 ,batch_size=batch_size
                                 ,tdNnum=tdNnum,tdNact=tdNact,sdCnum=sdCnum,sdCker=sdCker,sdCact=sdCact
                                 ,poolSize=poolSize,pooltype=pooltype
                                 ,LSnum=LSnum,LSdrop=LSdrop
                                 ,FNnum=FNnum,FNact=FNact,FNdrop=FNdrop,NUMCATEGORIES=NUMCATEGORIES
                                 ,optimizer='rmsprop'
                                 ,printModelName=1)

HaLT-filt830-Sub_G_interleaved_v4_09__tdNrelu_24_24_24__sdCrelu_30k5p3ave_20k5p3ave_15k5p3ave__GlobMaxpool__FCrelu_100_d0.5_50_d0.2_=6class__rmsprop_lr_0.0001_patience_150_50_factor_0.2_batch_256


In [22]:
import importlib
import interleavedV4
#from interleavedV4 import makeInterleavedModel, nameInterleavedModel
importlib.reload(interleavedV4)

model = makeInterleavedModel(input_shape
                             ,tdNnum,tdNact
                             ,sdCnum,sdCker,sdCact,poolSize,pooltype
                             ,LSnum,LSdrop,FNnum,FNact,FNdrop
                             ,NUMCATEGORIES,printModelSummary=1,printLayerShapes=1)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

(None, 65, 720)
(None, 20, 480)
(None, 5, 360)
ciao
(None, 5, 360)
(None, 360, 5)
ciao2
(None, 360, 5, 1)
(None, 360, 1)
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 200, 9)]          0         
_________________________________________________________________
tdNdense0 (TimeDistributed)  (None, 200, 24)           240       
_________________________________________________________________
tdNbanorm0 (BatchNormalizati (None, 200, 24)           96        
_________________________________________________________________
tdNact0 (Activation)         (None, 200, 24)           0         
_________________________________________________________________
permtd0 (Permute)            (None, 24, 200)           0         
_________________________________________________________________
reshtd0 (Reshape)            (None, 24, 200, 1)        0         
____